# 🦙 Fully Open-Source RAG in Google Colab


In [ ]:
# Install required libraries
!pip install -q llama-index-core llama-index-embeddings-huggingface \
                 llama-index-vector-stores-faiss transformers accelerate \
                 torch sentencepiece bitsandbytes

In [ ]:

# 1️⃣ Imports
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


In [ ]:
# 2️⃣ Load your local data (put some .txt files in /content/data)
data_path = "/content/data"
documents = SimpleDirectoryReader(data_path).load_data()

In [ ]:
# 3️⃣ Local embedding model (Hugging Face)
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# 4️⃣ Local FAISS vector store
faiss_store = FaissVectorStore.from_params(dim=384)
storage_context = StorageContext.from_defaults(vector_store=faiss_store)

In [ ]:
# 5️⃣ Build the vector index
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model
)

In [ ]:
# 6️⃣ Load an open-source LLM (via transformers)
# Recommended: small instruct model to fit in Colab GPU
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_8bit=True  # use less VRAM
)

In [ ]:
# 7️⃣ Query + Generate function (RAG)
def generate_response(query: str):
    # Retrieve top-K context chunks
    retriever = index.as_retriever(similarity_top_k=3)
    retrieved_docs = retriever.retrieve(query)
    context_text = "\n\n".join([d.get_text() for d in retrieved_docs])

    # Build final prompt for LLM
    prompt = (
        f"Context:\n{context_text}\n\n"
        f"Question: {query}\n\n"
        f"Answer concisely using the context above."
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# 8️⃣ Example usage
query = "How do I reset my device?"
print(generate_response(query))